In [1]:
# Make sure you have a GPU running
!nvidia-smi

Wed Jun 29 06:12:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install farm-haystack[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
"""
You might get a warning on executing the above cell for the first time due to some dependency issues
In that case please restart the runtime and run the above cell again otherwise the cells below won't get executed
"""

In [2]:
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.nodes import Seq2SeqGenerator

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [4]:
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

--2022-06-29 06:15:12--  https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23687259 (23M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.04.tar.gz’

xpdf-tools-linux-4. 100%[===================>]  22.59M  21.8MB/s    in 1.0s    

2022-06-29 06:15:14 (21.8 MB/s) - ‘xpdf-tools-linux-4.04.tar.gz’ saved [23687259/23687259]

xpdf-tools-linux-4.04/
xpdf-tools-linux-4.04/CHANGES
xpdf-tools-linux-4.04/COPYING3
xpdf-tools-linux-4.04/INSTALL
xpdf-tools-linux-4.04/ANNOUNCE
xpdf-tools-linux-4.04/bin64/
xpdf-tools-linux-4.04/bin64/pdftopng
xpdf-tools-linux-4.04/bin64/pdftohtml
xpdf-tools-linux-4.04/bin64/pdfinfo
xpdf-tools-linux-4.04/bin64/pdffonts
xpdf-tools-linux-4.04/bin64/pdfimages
xpdf-tools-linux-4.04/bin64/pdftotext
xpdf-tools-linux-4.04/bin64/pdftoppm
xpdf-tools-linux-4.04/bin64/pd

In [5]:
doc_dir="sample_data/files"

"""
Add the files to be used for fine-tuning to a folder in your colab notebook
For example here I created a folder named files in the default sample-data folder of my colab notebook
and I added the required files to this folder
"""

'\nAdd the files to be used for fine-tuning to a folder in your colab notebook\nFor example here I created a folder named files in the default sample-data folder of my colab notebook\nand I added the required files to this folder\n'

In [6]:
from haystack.utils.preprocessing import convert_files_to_docs
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting sample_data/files/BERT Research Paper_00001.txt


In [7]:
document_store.write_documents(docs)

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find vblagoje/dpr-ctx_encoder-single-lfqa-wiki locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded vblagoje/dpr-ctx_encoder-single-lfqa-wiki
INFO - haystack.document_stores.faiss -  Updating embeddings for 1 docs...


Updating Embedding:   0%|          | 0/1 [00:00<?, ? docs/s]

In [9]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [10]:
from haystack.pipelines import GenerativeQAPipeline
       
pipe = GenerativeQAPipeline(generator, retriever)

In [11]:
res=pipe.run(query="Who is Virat Kohli?", params={"Retriever": {"top_k": 3}})

In [28]:
answer=res['answers'][0].answer

In [ ]:
answer